In [5]:
!ls -la

total 24
drwxr-xr-x 1 root root 4096 Feb 24 16:56 .
drwxr-xr-x 1 root root 4096 Feb 24 16:56 ..
drwxr-xr-x 4 root root 4096 Feb 20 14:24 .config
-rw-r--r-- 1 root root 6757 Feb 24 16:56 google_queries.csv
drwxr-xr-x 1 root root 4096 Feb 20 14:24 sample_data


In [3]:
!cat .config

cat: .config: Is a directory


In [1]:
!head google_queries.csv

request,number_of_views,date
карта,3,2025-02-11
погода,5,2025-02-13
котики,5,2025-02-11
игры,2,2025-02-14
погода,1,2025-02-17
ютуб,1,2025-02-13
погода,1,2025-02-18
вк,1,2025-02-15
авито,5,2025-02-11


# Spark RDD
В **Apache Spark** существует интерфейс - RDD API. В нём производится работа с RDD напрямую.

<b>RDD </b>(resilent distrubuted dataset) - это фундаментальная структура данных Spark, которая представляет собой неизменяемый набор данных, который вычисляются и располагается на разных узлах кластера.

* [Guide](https://spark.apache.org/docs/latest/rdd-programming-guide.html)
* [Документация](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.RDD.html)

In [7]:
a = 1
b = 2

In [8]:
print(a+b)

3


In [6]:
!pip3 install pyspark

In [18]:
# Вариант 1. Создаем spark-сессию, явно указывая ее параметры
from pyspark.sql import SparkSession

from pyspark import SparkConf, SparkContext

"""
conf = (
    SparkConf()
        .set('spark.ui.port', '4050')
        .setMaster('local[*]')
)
sc = SparkContext(conf=conf)
spark = SparkSession(sc)
"""

ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=RDD_Example, master=local[*]) created by getOrCreate at <ipython-input-16-a9ab3538d9d1>:5 

In [19]:
# Вариант 2. Создаем дефолтную spark-сессию
from pyspark.sql import SparkSession

# Создае SparkSession
sc = SparkContext(conf=conf)
spark = SparkSession.builder.appName("RDD_Example").getOrCreate()

Для того, чтобы создать RDD необходимо к некоторой коллекции объектов применимеить операциюю parallelize. В результате работы spark разобъёт данные на куски (партиции) и отправит её части на разные worker ноды.

In [23]:
# Создаем RDD из списка
data = [("Иван", 30), ("Мария", 25), ("Алексей", 35), ("Елена", 40)]
rdd = spark.sparkContext.parallelize(data)

In [24]:
rdd

ParallelCollectionRDD[0] at readRDDFromFile at PythonRDD.scala:289

In [25]:
# Выведем данные RDD
print(rdd.collect())  # [('Иван', 30), ('Мария', 25), ('Алексей', 35), ('Елена', 40)]

[('Иван', 30), ('Мария', 25), ('Алексей', 35), ('Елена', 40)]


Посмотрим на количество партиций.

In [26]:
rdd.getNumPartitions()

2

Spark выбирает количество партиций для каждого RDD в зависимости от конфигруции кластера - по количеству доступных вычислительных ресурсов CPU. Но можно менять количество партиций, как в большую так и в меньшую сторону. Для того чтобы поменять количество партиций в существует два метода:

*   repartition - позволяет как увеличивать, так и уменьшать количество партиций, но производит полную перетасовку (shuffle) данных между узлами
*   coelesce - метод, с помощью которого можно только уменьшать размер партций. Не требует полной перетасовки данных между исполнителями, поэтому более эффективен.



In [27]:
repartitioned = rdd.repartition(5)
print(f"Num partitions after repartition: {repartitioned.getNumPartitions()}")

coelesced = repartitioned.coalesce(2)
print(f"Num partitions after coelesce: {coelesced.getNumPartitions()}")

Num partitions after repartition: 5
Num partitions after coelesce: 2


Обратной операцией к операции parallelize является метод collect, который наоборот создаёт коллекцию из данных, хранящихся на различных worker нодах.

In [28]:
rdd.collect()

[('Иван', 30), ('Мария', 25), ('Алексей', 35), ('Елена', 40)]

In [29]:
# К элементам RDD можно применять различные операции
# Допустим, мы хотим увеличить возраст каждого человека на 1 год
rdd_transformed = rdd.map(lambda x: (x[0], x[1] + 1))
print(rdd_transformed.collect())

[('Иван', 31), ('Мария', 26), ('Алексей', 36), ('Елена', 41)]


In [30]:
# Оставим только людей старше 30 лет
rdd_filtered = rdd.filter(lambda x: x[1] > 30)
print(rdd_filtered.collect())

[('Алексей', 35), ('Елена', 40)]


In [37]:
# Отсортируем людей во возрасту
rdd.sortByKey(keyfunc=lambda x: x[-1]).collect()

[('Елена', 40), ('Алексей', 35), ('Иван', 30), ('Мария', 25)]

In [38]:
# Отсортируем людей по именам
rdd.sortByKey().collect()

[('Алексей', 35), ('Елена', 40), ('Иван', 30), ('Мария', 25)]

In [39]:
# Допустим, у нас есть повторяющиеся имена, и мы хотим найти их средний возраст

data_with_duplicates = [("Иван", 30), ("Мария", 25), ("Иван", 40), ("Мария", 35)]
rdd_with_dup = spark.sparkContext.parallelize(data_with_duplicates)

In [40]:
rdd_with_dup.collect()

[('Иван', 30), ('Мария', 25), ('Иван', 40), ('Мария', 35)]

In [41]:
rdd_grouped = rdd_with_dup.groupByKey()

In [42]:
rdd_grouped.collect()

[('Иван', <pyspark.resultiterable.ResultIterable at 0x7e0f10c1ab90>),
 ('Мария', <pyspark.resultiterable.ResultIterable at 0x7e0f10c1bdd0>)]

In [43]:
[(key, list(values)) for key, values in rdd_grouped.collect()]

[('Иван', [30, 40]), ('Мария', [25, 35])]

In [44]:
# Хотим найти сумму возраста повторяющихся имен
rdd_with_dup.groupByKey().mapValues(sum).collect()

[('Иван', 70), ('Мария', 60)]

In [45]:
# Хотим найти количество повторяющихся имен
rdd_with_dup.groupByKey().mapValues(lambda x: len(list(x))).collect()

[('Иван', 2), ('Мария', 2)]

In [48]:
# Хотим найти средний возраст повторяющихся имен
rdd_with_dup.groupByKey().mapValues(lambda x: round(sum(list(x)) / len(list(x))) + 1).collect()

[('Иван', 36), ('Мария', 31)]

RDD API поддердивает прямую загрузку из текстового файла. При этом каждая строка будет интерпретироваться, как отдельный элемент RDD

In [49]:
! echo "Hello, sample RDD" > text.txt
! echo "This RDD contains three lines" >> text.txt
! echo "This is the last line" >> text.txt
! echo "" >> text.txt
! echo "Just kidding, it contains five lines" >> text.txt

In [50]:
!cat text.txt

Hello, sample RDD
This RDD contains three lines
This is the last line

Just kidding, it contains five lines


In [55]:
text_data = spark.sparkContext.textFile('text.txt')
text_data, text_data.collect()

(text.txt MapPartitionsRDD[84] at textFile at NativeMethodAccessorImpl.java:0,
 ['Hello, sample RDD',
  'This RDD contains three lines',
  'This is the last line',
  '',
  'Just kidding, it contains five lines'])

In [56]:
text_data.collect()

['Hello, sample RDD',
 'This RDD contains three lines',
 'This is the last line',
 '',
 'Just kidding, it contains five lines']

Теперь к данному RDD можно применять стандартные Spark операции

In [57]:
distinct_words = (
    text_data
        .filter(lambda x: len(x)) # отбираем только не пустые строки
        .flatMap(lambda x: x.split(' ')) # разбиваем все строки на слова и переводим список
        .distinct() # берём только уникальные слова
)

In [60]:
distinct_words

PythonRDD[89] at RDD at PythonRDD.scala:53

In [61]:
# Будьте внимательны, если такой файл существует,
# то spark будет выдавать ошибку
distinct_words.saveAsTextFile('words.txt')

In [62]:
# можно вывести отладочную информацию по данному RDD
print(distinct_words.toDebugString().decode())

(2) PythonRDD[89] at RDD at PythonRDD.scala:53 []
 |  MapPartitionsRDD[88] at mapPartitions at PythonRDD.scala:160 []
 |  ShuffledRDD[87] at partitionBy at NativeMethodAccessorImpl.java:0 []
 +-(2) PairwiseRDD[86] at distinct at <ipython-input-57-307dbd238947>:5 []
    |  PythonRDD[85] at distinct at <ipython-input-57-307dbd238947>:5 []
    |  text.txt MapPartitionsRDD[84] at textFile at NativeMethodAccessorImpl.java:0 []
    |  text.txt HadoopRDD[83] at textFile at NativeMethodAccessorImpl.java:0 []


Снизу вверх показаны все низкоуровневые операции (lineage), которые были применены к данному RDD c самого начала его создания

Для того, чтобы переиспользовать посчитанные значения в рамках текущей сессии стоит использовать метод `.cache`, который сохраняет результат вычислений вершины графа вычислений в оперативной памяти. Это нужно для того, чтобы оперции, работающие поверх данной получали результат операций из оперативной памяти, а не считывались с диска.

Метод `.persist` позволяет сохранять промежуточные вычисления в рамках текущей сессии с более тонкой настройкой места хранения (жёсткий диск, оперативная память, ...)

In [63]:
distinct_words_cached = distinct_words.cache()
print(distinct_words_cached.toDebugString().decode())

(2) PythonRDD[89] at RDD at PythonRDD.scala:53 [Memory Serialized 1x Replicated]
 |  MapPartitionsRDD[88] at mapPartitions at PythonRDD.scala:160 [Memory Serialized 1x Replicated]
 |  ShuffledRDD[87] at partitionBy at NativeMethodAccessorImpl.java:0 [Memory Serialized 1x Replicated]
 +-(2) PairwiseRDD[86] at distinct at <ipython-input-57-307dbd238947>:5 [Memory Serialized 1x Replicated]
    |  PythonRDD[85] at distinct at <ipython-input-57-307dbd238947>:5 [Memory Serialized 1x Replicated]
    |  text.txt MapPartitionsRDD[84] at textFile at NativeMethodAccessorImpl.java:0 [Memory Serialized 1x Replicated]
    |  text.txt HadoopRDD[83] at textFile at NativeMethodAccessorImpl.java:0 [Memory Serialized 1x Replicated]


Как можно заметить после операции cache появились дополнительные вершины в графе вычислений, в которых указаны место расположения данных и количество их реплик: Memory Serialized 1x Replicated

In [64]:
distinct_words_cached.collect()
print(distinct_words_cached.toDebugString().decode())

(2) PythonRDD[89] at RDD at PythonRDD.scala:53 [Memory Serialized 1x Replicated]
 |       CachedPartitions: 2; MemorySize: 296.0 B; DiskSize: 0.0 B
 |  MapPartitionsRDD[88] at mapPartitions at PythonRDD.scala:160 [Memory Serialized 1x Replicated]
 |  ShuffledRDD[87] at partitionBy at NativeMethodAccessorImpl.java:0 [Memory Serialized 1x Replicated]
 +-(2) PairwiseRDD[86] at distinct at <ipython-input-57-307dbd238947>:5 [Memory Serialized 1x Replicated]
    |  PythonRDD[85] at distinct at <ipython-input-57-307dbd238947>:5 [Memory Serialized 1x Replicated]
    |  text.txt MapPartitionsRDD[84] at textFile at NativeMethodAccessorImpl.java:0 [Memory Serialized 1x Replicated]
    |  text.txt HadoopRDD[83] at textFile at NativeMethodAccessorImpl.java:0 [Memory Serialized 1x Replicated]


Для сохранения данных между сессиями можно использовать `.checkpoint`. Особенность этого метода — изменение графа вычислений.
Цепочка вычислений для сохраняемого RDD будет удалена.

Сокращение цепочки вычислений полезно в случае больших графов, например, в итеративных алгоритмах.

In [66]:
distinct_first_words = (
    text_data
        .filter(lambda x: len(x))
        .flatMap(lambda x: x.split(' ')[0])
        .distinct()
)

spark.sparkContext.setCheckpointDir('./checkpoints')

distinct_first_words.checkpoint()
print(distinct_first_words.toDebugString().decode())

(2) PythonRDD[101] at RDD at PythonRDD.scala:53 []
 |  MapPartitionsRDD[100] at mapPartitions at PythonRDD.scala:160 []
 |  ShuffledRDD[99] at partitionBy at NativeMethodAccessorImpl.java:0 []
 +-(2) PairwiseRDD[98] at distinct at <ipython-input-66-dc207dcbce62>:5 []
    |  PythonRDD[97] at distinct at <ipython-input-66-dc207dcbce62>:5 []
    |  text.txt MapPartitionsRDD[84] at textFile at NativeMethodAccessorImpl.java:0 []
    |  text.txt HadoopRDD[83] at textFile at NativeMethodAccessorImpl.java:0 []


In [68]:
!ls -la ./checkpoints/d1826eb9-af1e-4ea1-af0a-53f441ba148a

total 8
drwxr-xr-x 2 root root 4096 Feb 24 17:34 .
drwxr-xr-x 3 root root 4096 Feb 24 17:34 ..


In [69]:
distinct_first_words.collect()
print(distinct_first_words.toDebugString().decode())

(2) PythonRDD[101] at RDD at PythonRDD.scala:53 []
 |  ReliableCheckpointRDD[102] at collect at <ipython-input-69-c1aaa7e99388>:1 []


Как можно увидеть предыдущий граф вычилений был полность удалён, и теперь вычисления начинаются с загрузки контрольной точки: ReliableCheckpointRDD

# Практика

Есть файл google_queries.csv, в нем указаны запросы пользователей в Google, количесво данных запросов в день, день запроса

In [ ]:
!head google_queries.csv

request,number_of_views,date
карта,3,2025-02-11
погода,5,2025-02-13
котики,5,2025-02-11
игры,2,2025-02-14
погода,1,2025-02-17
ютуб,1,2025-02-13
погода,1,2025-02-18
вк,1,2025-02-15
авито,5,2025-02-11


In [70]:
from pyspark.sql import SparkSession

# Создаем SparkSession
spark = SparkSession.builder.appName("GoogleQueriesAnalysis").getOrCreate()
sc = spark.sparkContext  # Получаем SparkContext

# Загружаем файл в RDD
rdd = sc.textFile("google_queries.csv")

In [71]:
rdd.take(5)

['request,number_of_views,date',
 'карта,3,2025-02-11',
 'погода,5,2025-02-13',
 'котики,5,2025-02-11',
 'игры,2,2025-02-14']

In [72]:
rdd.first()

'request,number_of_views,date'

In [73]:
# Пропускаем заголовок
header = rdd.first()
rdd = rdd.filter(lambda line: line != header)

In [74]:
rdd.take(5)

['карта,3,2025-02-11',
 'погода,5,2025-02-13',
 'котики,5,2025-02-11',
 'игры,2,2025-02-14',
 'погода,1,2025-02-17']

In [75]:
rdd.map(lambda line: line.split(",")).take(5)

[['карта', '3', '2025-02-11'],
 ['погода', '5', '2025-02-13'],
 ['котики', '5', '2025-02-11'],
 ['игры', '2', '2025-02-14'],
 ['погода', '1', '2025-02-17']]

In [76]:
# Парсим данные (request, number_of_views, date)
rdd_parsed = rdd.map(lambda line: line.split(",")).map(lambda x: (x[0], int(x[1]), x[2]))

In [77]:
rdd_parsed.take(5)

[('карта', 3, '2025-02-11'),
 ('погода', 5, '2025-02-13'),
 ('котики', 5, '2025-02-11'),
 ('игры', 2, '2025-02-14'),
 ('погода', 1, '2025-02-17')]

In [78]:
# 1. Общее количество запросов
total_queries = rdd_parsed.count()
print(f"1. Общее количество запросов: {total_queries}")

1. Общее количество запросов: 255


In [79]:
# 2. Общее количество просмотров
total_views = rdd_parsed.map(lambda x: x[1]).sum()
print(f"2. Общее количество просмотров: {total_views}")

2. Общее количество просмотров: 743


In [80]:
rdd_parsed.take(5)

[('карта', 3, '2025-02-11'),
 ('погода', 5, '2025-02-13'),
 ('котики', 5, '2025-02-11'),
 ('игры', 2, '2025-02-14'),
 ('погода', 1, '2025-02-17')]

In [81]:
# 3. Среднее количество просмотров на запрос
avg_views_per_query = total_views / total_queries if total_queries > 0 else 0
print(f"3. Среднее количество просмотров на запрос: {avg_views_per_query:.2f}")

3. Среднее количество просмотров на запрос: 2.91


In [83]:
# 4. Запрос с максимальным числом просмотров
max_request = rdd_parsed.max(key=lambda x: x[1])
print(f"4. Запрос с максимальным числом просмотров: {max_request}")

4. Запрос с максимальным числом просмотров: ('погода', 5, '2025-02-13')


In [84]:
# 5. Запрос с минимальным числом просмотров
min_request = rdd_parsed.min(key=lambda x: x[1])
print(f"5. Запрос с минимальным числом просмотров: {min_request}")

5. Запрос с минимальным числом просмотров: ('погода', 1, '2025-02-17')


In [85]:
rdd_parsed.take(5)

[('карта', 3, '2025-02-11'),
 ('погода', 5, '2025-02-13'),
 ('котики', 5, '2025-02-11'),
 ('игры', 2, '2025-02-14'),
 ('погода', 1, '2025-02-17')]

In [93]:
# 6. Количество уникальных запросов
unique_requests = rdd_parsed.map(lambda x: x[0]).distinct().count()
print(f"6. Количество уникальных запросов: {unique_requests}")

6. Количество уникальных запросов: 14


In [105]:
rdd_parsed.map(lambda x: (x[2], x[1])).groupByKey().take(5)

[('2025-02-11', <pyspark.resultiterable.ResultIterable at 0x7e0f0f23ac10>),
 ('2025-02-13', <pyspark.resultiterable.ResultIterable at 0x7e0f10c58550>),
 ('2025-02-19', <pyspark.resultiterable.ResultIterable at 0x7e0f0f1be250>),
 ('2025-02-12', <pyspark.resultiterable.ResultIterable at 0x7e0f0f1bc910>),
 ('2025-02-14', <pyspark.resultiterable.ResultIterable at 0x7e0f0f26a850>)]

In [97]:
rdd_parsed.map(lambda x: (x[2], x[1])).groupByKey().mapValues(lambda values: sum(values) / len(values)).take(5)

[('2025-02-11', 2.7714285714285714),
 ('2025-02-13', 2.92),
 ('2025-02-19', 3.033333333333333),
 ('2025-02-12', 2.6956521739130435),
 ('2025-02-14', 3.1)]

In [98]:
# 7. Среднее количество просмотров на дату
views_per_date = rdd_parsed.map(lambda x: (x[2], x[1])).groupByKey().mapValues(lambda values: sum(values) / len(values))
avg_views_per_date = views_per_date.map(lambda x: x[1]).mean()
print(f"7. Среднее количество просмотров на дату: {avg_views_per_date:.2f}")

7. Среднее количество просмотров на дату: 2.92


In [106]:
rdd_parsed.take(5)

[('карта', 3, '2025-02-11'),
 ('погода', 5, '2025-02-13'),
 ('котики', 5, '2025-02-11'),
 ('игры', 2, '2025-02-14'),
 ('погода', 1, '2025-02-17')]

In [109]:
rdd_parsed.map(lambda x: x[2]).collect()

['2025-02-11',
 '2025-02-13',
 '2025-02-11',
 '2025-02-14',
 '2025-02-17',
 '2025-02-13',
 '2025-02-18',
 '2025-02-15',
 '2025-02-11',
 '2025-02-15',
 '2025-02-13',
 '2025-02-17',
 '2025-02-15',
 '2025-02-16',
 '2025-02-10',
 '2025-02-13',
 '2025-02-19',
 '2025-02-11',
 '2025-02-13',
 '2025-02-10',
 '2025-02-15',
 '2025-02-14',
 '2025-02-13',
 '2025-02-11',
 '2025-02-11',
 '2025-02-12',
 '2025-02-19',
 '2025-02-18',
 '2025-02-16',
 '2025-02-18',
 '2025-02-10',
 '2025-02-12',
 '2025-02-11',
 '2025-02-17',
 '2025-02-15',
 '2025-02-14',
 '2025-02-13',
 '2025-02-15',
 '2025-02-13',
 '2025-02-14',
 '2025-02-17',
 '2025-02-11',
 '2025-02-15',
 '2025-02-17',
 '2025-02-16',
 '2025-02-13',
 '2025-02-10',
 '2025-02-12',
 '2025-02-17',
 '2025-02-12',
 '2025-02-18',
 '2025-02-17',
 '2025-02-16',
 '2025-02-17',
 '2025-02-10',
 '2025-02-19',
 '2025-02-17',
 '2025-02-14',
 '2025-02-14',
 '2025-02-12',
 '2025-02-10',
 '2025-02-17',
 '2025-02-11',
 '2025-02-17',
 '2025-02-11',
 '2025-02-11',
 '2025-02-

In [110]:
rdd_parsed.map(lambda x: x[2]).distinct().collect()

['2025-02-11',
 '2025-02-13',
 '2025-02-19',
 '2025-02-12',
 '2025-02-14',
 '2025-02-17',
 '2025-02-18',
 '2025-02-15',
 '2025-02-16',
 '2025-02-10']

In [111]:
# 8. Общее число дат, когда были поисковые запросы
unique_dates = rdd_parsed.map(lambda x: x[2]).distinct().count()
print(f"8. Общее число дат, когда были поисковые запросы: {unique_dates}")

8. Общее число дат, когда были поисковые запросы: 10


In [112]:
rdd_parsed.take(5)

[('карта', 3, '2025-02-11'),
 ('погода', 5, '2025-02-13'),
 ('котики', 5, '2025-02-11'),
 ('игры', 2, '2025-02-14'),
 ('погода', 1, '2025-02-17')]



1.   groupBy
2.   groupByKey
3.   reduceByKey



In [114]:
rdd_parsed.map(lambda x: (x[0], x[2])).distinct().map(lambda x: (x[0], 1)) \
  .reduceByKey(lambda a, b: a + b).take(5)

[('ютуб', 9), ('спорт', 8), ('игры', 8), ('музыка', 9), ('фильмы', 10)]

In [115]:
# 9. Запрос с наибольшим числом дней появления
query_day_count = rdd_parsed.map(lambda x: (x[0], x[2])).distinct().map(lambda x: (x[0], 1)) \
                            .reduceByKey(lambda a, b: a + b)
max_days_request = query_day_count.max(key=lambda x: x[1])
print(f"9. Запрос с наибольшим числом дней появления: {max_days_request}")

9. Запрос с наибольшим числом дней появления: ('фильмы', 10)


In [116]:
rdd_parsed.take(5)

[('карта', 3, '2025-02-11'),
 ('погода', 5, '2025-02-13'),
 ('котики', 5, '2025-02-11'),
 ('игры', 2, '2025-02-14'),
 ('погода', 1, '2025-02-17')]

In [117]:
rdd_parsed.map(lambda x: (x[2], x[1])).reduceByKey(lambda a, b: a + b).take(5)

[('2025-02-11', 97),
 ('2025-02-13', 73),
 ('2025-02-19', 91),
 ('2025-02-12', 62),
 ('2025-02-14', 62)]

In [118]:
# 10. Самая популярная дата по просмотрам
popular_date = rdd_parsed.map(lambda x: (x[2], x[1])).reduceByKey(lambda a, b: a + b) \
                         .max(key=lambda x: x[1])
print(f"10. Самая популярная дата по просмотрам: {popular_date}")

10. Самая популярная дата по просмотрам: ('2025-02-11', 97)


In [119]:
rdd_parsed.take(5)

[('карта', 3, '2025-02-11'),
 ('погода', 5, '2025-02-13'),
 ('котики', 5, '2025-02-11'),
 ('игры', 2, '2025-02-14'),
 ('погода', 1, '2025-02-17')]

In [126]:
# 11. Дата с наибольшим количеством уникальных запросов
unique_queries_per_date = rdd_parsed.map(lambda x: (x[2], x[0])).distinct() \
                                    .map(lambda x: (x[0], 1)).reduceByKey(lambda a, b: a + b)
max_unique_query_date = unique_queries_per_date.max(key=lambda x: x[1])
print(f"11. Дата с наибольшим количеством уникальных запросов: {max_unique_query_date}")

11. Дата с наибольшим количеством уникальных запросов: ('2025-02-11', 14)


In [131]:
# 11. Дата с наибольшим количеством уникальных запросов
unique_queries_per_date = rdd_parsed.map(lambda x: (x[2], x[0])).distinct() \
                                    .map(lambda x: (x[0], 1)).reduceByKey(lambda a, b: a + b)
max_unique_query_date = unique_queries_per_date.min(key=lambda x: x[1])
print(f"11. Дата с наибольшим количеством уникальных запросов: {max_unique_query_date}")

11. Дата с наибольшим количеством уникальных запросов: ('2025-02-18', 9)


In [128]:
rdd_parsed.take(5)

[('карта', 3, '2025-02-11'),
 ('погода', 5, '2025-02-13'),
 ('котики', 5, '2025-02-11'),
 ('игры', 2, '2025-02-14'),
 ('погода', 1, '2025-02-17')]

In [130]:
unique_queries_per_date.count()

10

In [132]:
# 12. Общее количество дней, когда было более 3 уникальных запросов
days_with_more_than_3_queries = unique_queries_per_date.filter(lambda x: x[1] > 10).count()
print(f"12. Общее количество дней с >3 уникальными запросами: {days_with_more_than_3_queries}")

12. Общее количество дней с >3 уникальными запросами: 8


In [133]:
rdd_parsed.take(5)

[('карта', 3, '2025-02-11'),
 ('погода', 5, '2025-02-13'),
 ('котики', 5, '2025-02-11'),
 ('игры', 2, '2025-02-14'),
 ('погода', 1, '2025-02-17')]

In [134]:
# 13. Топ-3 самых популярных запроса по просмотрам
top_3_queries = rdd_parsed.map(lambda x: (x[0], x[1])).reduceByKey(lambda a, b: a + b) \
                          .top(3, key=lambda x: x[1])
print(f"13. Топ-3 самых популярных запроса по просмотрам: {top_3_queries}")

13. Топ-3 самых популярных запроса по просмотрам: [('игры', 68), ('авито', 65), ('вк', 61)]


In [149]:
rdd_parsed.map(lambda x: (x[0], x[1])).reduceByKey(lambda a, b: a + b) \
  .map(lambda x: (x[1], x[0])) \
  .sortByKey().take(3)

[(33, 'котики'), (35, 'вконтакте'), (38, 'новости')]